In [11]:
import json
import torch
import os
import numpy as np
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from sentence_transformers import SentenceTransformer
from torch.optim import Adam
from torch.nn.functional import cosine_similarity
from datasets import load_dataset

In [12]:
# clear cuda cache
torch.cuda.empty_cache()
# clear cuda memory
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()

In [13]:
device = torch.device("cpu")

# Initialize a Sentence-BERT model for multi-hop retrieval and reward computation.
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

In [14]:
# RL and training hyperparameters
LEARNING_RATE = 5e-5
num_epochs = 2          # For demonstration; use more epochs in practice.
temperature = 0.8
max_new_tokens = 64     # Maximum tokens to sample per answer
alpha = 0.5             # Weight for supervised loss vs. RL loss.


In [15]:
def compute_reward(model_answer, gold_answer):
    """
    Compute a reward by comparing the cosine similarity between
    embeddings of the model's answer and the gold answer.
    """
    model_embedding = retriever_model.encode(model_answer, convert_to_tensor=True)
    gold_embedding = retriever_model.encode(gold_answer, convert_to_tensor=True)
    if len(model_embedding.shape) == 1:
        model_embedding = model_embedding.unsqueeze(0)
    if len(gold_embedding.shape) == 1:
        gold_embedding = gold_embedding.unsqueeze(0)
    # Higher similarity gives a higher reward.
    return cosine_similarity(model_embedding, gold_embedding, dim=1).item()

In [16]:
def save_list_to_json(lst, filename):
    with open(filename, 'w') as file:
        json.dump(lst, file)

def rm_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File {file_path} removed successfully.")

In [17]:
# Load model and tokenizer.
model_name = "microsoft/DialoGPT-medium"
save_file = 'qa_output/mistral_rl_triviaqa.json'
# Here we use float32 for stability.
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from rank_bm25 import BM25Okapi
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer

# Load DPR models and tokenizers
dpr_question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
dpr_context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
dpr_context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

In [18]:
# Fixed prompt prefix.
prefix = (
    "Below is a question followed by context from different sources. "
    "Please answer using the context. If insufficient, respond 'Insufficient Information'. "
    "Answer directly."
)


In [19]:
# Load a small subset of the TriviaQA dataset.
dataset = load_dataset("trivia_qa", "rc", split="train[:100]")
doc_data = []
for example in dataset:
    query = example["question"]
    # Use available evidence if present; otherwise, provide a dummy context.
    if "evidence" in example and len(example["evidence"]) > 0:
        retrieval_list = [{"text": example["evidence"][0]["text"]}]
    else:
        retrieval_list = [{"text": "No additional context available."}]
    # Assume the answer is stored as a string or in a dict.
    answer = example["answer"]["value"] if isinstance(example["answer"], dict) else example["answer"]
    doc_data.append({
        "query": query,
        "retrieval_list": retrieval_list,
        "answer": answer,
        "question_type": "trivia"
    })

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

In [20]:
# RL Optimizer.
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

def retrieve_context(query, retrieval_list, hops=2, alpha=0.5):
    """
    Hybrid retrieval using BM25 + DPR. 
    - alpha: Weight between BM25 (lexical) and DPR (semantic).
    """
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    
    # BM25 setup
    corpus = [doc["text"] for doc in retrieval_list]
    tokenized_corpus = [doc.split() for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    
    # BM25 scores
    bm25_scores = bm25.get_scores(query.split())

    # DPR scores
    with torch.no_grad():
        dpr_query_emb = dpr_question_encoder(**dpr_question_tokenizer(query, return_tensors="pt").to(device)).pooler_output
        dpr_doc_embs = []
        for doc in corpus:
            doc_emb = dpr_context_encoder(**dpr_context_tokenizer(doc, return_tensors="pt").to(device)).pooler_output
            dpr_doc_embs.append(doc_emb)
        dpr_doc_embs = torch.cat(dpr_doc_embs)
        dpr_scores = torch.nn.functional.cosine_similarity(dpr_query_emb, dpr_doc_embs).cpu().numpy()

    # Combine scores (weighted sum)
    combined_scores = alpha * np.array(bm25_scores) + (1 - alpha) * np.array(dpr_scores)

    # Rank documents
    ranked_indices = np.argsort(combined_scores)[::-1]
    context = "\n\n".join([corpus[i] for i in ranked_indices[:hops]])

    return context

In [21]:
def query_bot_rl(prompt, temperature=temperature, max_new_tokens=max_new_tokens):
    """
    Generate an answer token-by-token using sampling, while accumulating log probabilities.
    Returns the generated text and total log probability.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    generated_tokens = []
    log_probs = []
    for _ in range(max_new_tokens):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # Convert logits to float32 to avoid NaN issues.
        logits = outputs.logits[:, -1, :].to(torch.float32)
        logits = logits / temperature
        distribution = torch.distributions.Categorical(logits=logits)
        token = distribution.sample()  # Sample next token.
        log_prob = distribution.log_prob(token)
        generated_tokens.append(token)
        log_probs.append(log_prob)
        # Append the sampled token to the input sequence.
        input_ids = torch.cat([input_ids, token.unsqueeze(-1)], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones_like(token.unsqueeze(-1))], dim=-1)
        # Stop if EOS token is generated.
        if token.item() == tokenizer.eos_token_id:
            break
    total_log_prob = torch.stack(log_probs).sum()
    generated_text = tokenizer.decode(torch.cat(generated_tokens), skip_special_tokens=True)
    return generated_text, total_log_prob

In [ ]:
# Training Loop with Hybrid Training on TriviaQA subset.
rm_file(save_file)
save_list = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for d in tqdm(doc_data):
        # Retrieve context and construct prompt.
        retrieval_list = d['retrieval_list']
        context = retrieve_context(d['query'], retrieval_list, hops=2)
        prompt = f"{prefix}\n\nQuestion: {d['query']}\n\nContext:\n\n{context}"
        
        model.train()

        # ----- RL Branch -----
        # Generate an answer using RL sampling.
        rl_response, total_log_prob = query_bot_rl(prompt)
        # Compute reward comparing the generated answer with the gold answer.
        reward = compute_reward(rl_response, d['answer'])
        reward_tensor = torch.tensor(reward, device=device, dtype=total_log_prob.dtype)
        rl_loss = -reward_tensor * total_log_prob

        # ----- Supervised (MLE) Branch -----
        # Construct a combined input: prompt followed by the gold answer.
        # We add a marker "Answer:" to indicate the beginning of the answer.
        prompt_text = prompt + "\n\nAnswer:"
        prompt_enc = tokenizer(prompt_text, return_tensors="pt").to(device)
        answer_enc = tokenizer(d['answer'], return_tensors="pt").to(device)
        # Concatenate the prompt and gold answer.
        input_ids = torch.cat([prompt_enc.input_ids, answer_enc.input_ids], dim=1)
        attention_mask = torch.cat([prompt_enc.attention_mask, answer_enc.attention_mask], dim=1)
        # Create labels: ignore loss for prompt tokens.
        labels = input_ids.clone()
        labels[:, :prompt_enc.input_ids.shape[1]] = -100

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        supervised_loss = outputs.loss

        # ----- Hybrid Loss -----
        # Combine supervised (MLE) loss and RL loss.
        combined_loss = alpha * supervised_loss + (1 - alpha) * rl_loss

        optimizer.zero_grad()
        combined_loss.backward()
        optimizer.step()

        # Save training metrics.
        save = {
            'query': d['query'],
            'prompt': prompt,
            'rl_model_answer': rl_response,
            'gold_answer': d['answer'],
            'question_type': d['question_type'],
            'reward': reward,
            'supervised_loss': supervised_loss.item(),
            'rl_loss': rl_loss.item(),
            'combined_loss': combined_loss.item()
        }
        save_list.append(save)
        
    # Optionally, save a checkpoint after each epoch.
    torch.save(model.state_dict(), f"checkpoint_dpr_multi_rl_epoch_{epoch+1}.pt")
    print(f"Epoch {epoch+1} completed.")

save_list_to_json(save_list, save_file)


Epoch 1/2


100%|██████████| 100/100 [05:06<00:00,  3.06s/it]


Epoch 1 completed.
Epoch 2/2


100%|██████████| 100/100 [05:04<00:00,  3.04s/it]


Epoch 2 completed.


In [22]:
model.load_state_dict(torch.load("checkpoint_dpr_multi_rl_epoch_4.pt"))
# Training Loop with Hybrid Training on TriviaQA subset.
save_list = []
epoch = 4
for d in tqdm(doc_data):
    # Retrieve context and construct prompt.
    retrieval_list = d['retrieval_list']
    context = retrieve_context(d['query'], retrieval_list, hops=2)
    prompt = f"{prefix}\n\nQuestion: {d['query']}\n\nContext:\n\n{context}"
    
    model.train()

    # ----- RL Branch -----
    # Generate an answer using RL sampling.
    rl_response, total_log_prob = query_bot_rl(prompt)
    # Compute reward comparing the generated answer with the gold answer.
    reward = compute_reward(rl_response, d['answer'])
    reward_tensor = torch.tensor(reward, device=device, dtype=total_log_prob.dtype)
    rl_loss = -reward_tensor * total_log_prob

    # ----- Supervised (MLE) Branch -----
    # Construct a combined input: prompt followed by the gold answer.
    # We add a marker "Answer:" to indicate the beginning of the answer.
    prompt_text = prompt + "\n\nAnswer:"
    prompt_enc = tokenizer(prompt_text, return_tensors="pt").to(device)
    answer_enc = tokenizer(d['answer'], return_tensors="pt").to(device)
    # Concatenate the prompt and gold answer.
    input_ids = torch.cat([prompt_enc.input_ids, answer_enc.input_ids], dim=1)
    attention_mask = torch.cat([prompt_enc.attention_mask, answer_enc.attention_mask], dim=1)
    # Create labels: ignore loss for prompt tokens.
    labels = input_ids.clone()
    labels[:, :prompt_enc.input_ids.shape[1]] = -100

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    supervised_loss = outputs.loss

    # ----- Hybrid Loss -----
    # Combine supervised (MLE) loss and RL loss.
    combined_loss = alpha * supervised_loss + (1 - alpha) * rl_loss

    optimizer.zero_grad()
    combined_loss.backward()
    optimizer.step()

    # Save training metrics.
    save = {
        'query': d['query'],
        'prompt': prompt,
        'rl_model_answer': rl_response,
        'gold_answer': d['answer'],
        'question_type': d['question_type'],
        'reward': reward,
        'supervised_loss': supervised_loss.item(),
        'rl_loss': rl_loss.item(),
        'combined_loss': combined_loss.item()
    }
    save_list.append(save)
    
# Optionally, save a checkpoint after each epoch.
torch.save(model.state_dict(), f"checkpoint_dpr_multi_rl_epoch_{epoch+1}.pt")
print(f"Epoch {epoch+1} completed.")
save_list_to_json(save_list, save_file)


100%|██████████| 100/100 [04:49<00:00,  2.89s/it]


Epoch 5 completed.


In [26]:
from evaluate import load

test_data = load_dataset("trivia_qa", "rc", split="test[:100]")

doc_data = []
for example in test_data:
    query = example["question"]
    # Use available evidence if present; otherwise, provide a dummy context.
    if "evidence" in example and len(example["evidence"]) > 0:
        retrieval_list = [{"text": example["evidence"][0]["text"]}]
    else:
        retrieval_list = [{"text": "No additional context available."}]
    # Assume the answer is stored as a string or in a dict.
    answer = example["answer"]["value"] if isinstance(example["answer"], dict) else example["answer"]
    doc_data.append({
        "query": query,
        "retrieval_list": retrieval_list,
        "answer": answer,
        "question_type": "trivia"
    })

# Load evaluation metrics
bleu = load("bleu")
rouge = load("rouge")
bert_score = load("bertscore")

# Generate predictions
predictions = []
ground_truths = []

D = []

for data in tqdm(doc_data):
    retrieval_list = data['retrieval_list']
    context = retrieve_context(data['query'], retrieval_list, hops=2)
    prompt = f"{prefix}\n\nQuestion: {data['query']}\n\nContext:\n\n{context}"
    gold_answer = data['answer']
    model_answer, _ = query_bot_rl(prompt)

    predictions.append(model_answer)
    ground_truths.append(gold_answer)
    
    reward = compute_reward(model_answer, gold_answer)
    D.append(reward)

# Compute Exact Match (EM)
exact_match = sum([1 if pred.lower().strip() == gold.lower().strip() else 0 
                   for pred, gold in zip(predictions, ground_truths)]) / len(ground_truths)

# Compute BLEU Score
bleu_score = bleu.compute(predictions=predictions, references=ground_truths)

# Compute ROUGE Score
rouge_score = rouge.compute(predictions=predictions, references=ground_truths)

# Compute BERTScore
bert_score_result = bert_score.compute(predictions=predictions, references=ground_truths, lang="en")
avg_bert_score = sum(bert_score_result['f1']) / len(bert_score_result['f1'])

# Compute average reward
avg_reward = np.mean(D)

# Print Results
print(f"Exact Match (EM): {exact_match:.4f}")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print(f"ROUGE-L Score: {rouge_score['rougeL']:.4f}")
print(f"BERTScore (F1): {sum(bert_score_result['f1']) / len(bert_score_result['f1']):.4f}")
print(f"Average Reward: {avg_reward:.4f}")

print("Evaluation completed.")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Exact Match (EM): 0.0000
BLEU Score: 0.0000
ROUGE-L Score: 0.0000
BERTScore (F1): 0.9007
Average Reward: 0.1622
Evaluation completed.
